**IN QGIS**

 - reproject raster to 4326
 - simple reclass with -2147483647 (No Data value) to 193 (Value used by USDA to determine rural 500 people/square mile) to -99999 (no data value default for tiffs) and >= 193 is 1
 - Polygonize the resulting tif with polygonize tool in qgis.  Below is the same option with python bindings

In [ ]:
from osgeo import gdal, ogr
import sys
# this allows GDAL to throw Python Exceptions
gdal.UseExceptions()

#
#  get raster datasource
#
from os.path import expanduser
import os.path as op
home = expanduser("~")

src_ds = gdal.Open( op.join(home,"downloads/Landscan/landscanworld.tif" )
if src_ds is None:
    print 'Unable to open %s' % src_filename
    sys.exit(1)

try:
    srcband = src_ds.GetRasterBand(3)
except RuntimeError, e:
    # for example, try GetRasterBand(10)
    print 'Band ( %i ) not found' % band_num
    print e
    sys.exit(1)

#
#  create output datasource
#
dst_layername = "POLYGONIZED_STUFF"
drv = ogr.GetDriverByName("ESRI Shapefile")
dst_ds = drv.CreateDataSource( dst_layername + ".shp" )
dst_layer = dst_ds.CreateLayer(dst_layername, srs = None )

gdal.Polygonize( srcband, None, dst_layer, -1, [], callback=None )